In [1]:
print("ok")

ok


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [2]:
PINECONE_API_KEY = '1443d596-266d-4c34-83f6-e8903fc4c6ff'


In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [6]:
extracted_data

[Document(page_content='', metadata={'source': 'data\\Medical_book.pdf', 'page': 0}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1', metadata={'source': 'data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Content\nKelly 

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk: ", len(text_chunks))

length of my chunk:  7020


In [93]:
#text_chunks

In [9]:
#download embedding model

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embedding()

c:\Users\rites\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length: ", len(query_result))

Length:  384


In [13]:
query_result

[-0.03447723761200905,
 0.031023172661662102,
 0.006734967231750488,
 0.02610894665122032,
 -0.03936203941702843,
 -0.1603025197982788,
 0.06692396849393845,
 -0.006441415287554264,
 -0.0474504753947258,
 0.014758882112801075,
 0.07087530195713043,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625138871371746,
 -0.010109545662999153,
 -0.026940535753965378,
 0.02230738289654255,
 -0.022226667031645775,
 -0.14969263970851898,
 -0.01749306730926037,
 0.007676335051655769,
 0.05435227230191231,
 0.003254442475736141,
 0.03172587975859642,
 -0.08462145179510117,
 -0.02940605953335762,
 0.05159563571214676,
 0.04812406376004219,
 -0.0033148345537483692,
 -0.058279212564229965,
 0.04196925461292267,
 0.022210633382201195,
 0.1281888782978058,
 -0.02233893983066082,
 -0.011656169779598713,
 0.06292835623025894,
 -0.03287630155682564,
 -0.09122607856988907,
 -0.031175361946225166,
 0.05269959196448326,
 0.04703481122851372,
 -0.0842030867934227,
 -0.030056176707148552,
 -0.020744783803820

In [16]:
PINECONE_API_ENV = 'gcp-starter'

In [15]:
#Initializing the Pinecone

from langchain_pinecone import PineconeVectorStore
import os

os.environ["PINECONE_API_KEY"] = "1443d596-266d-4c34-83f6-e8903fc4c6ff"

pinecone_vector_store = PineconeVectorStore(index_name="medical-chatbot", embedding=embeddings)

pinecone_vector_store.add_texts(
    [doc.page_content for doc in text_chunks],  # Extract page_content from Document objects
    metadatas=[{"source": "data\\Medical_book.pdf", "page": i} for i in range(len(text_chunks))],
)


In [16]:
#If we already have an index we can load it like this
query = "Medications for fever"
docs = pinecone_vector_store.similarity_search(query, k=3)

print("Result:")
for doc in docs:
    print(f"Text: {doc.page_content}")
    # if doc.metadata:
    #     print(f"Source: {doc.metadata.get('source', 'N/A')}, Page: {doc.metadata.get('page', 'N/A')}")
    print("-" * 20)

Result:
Text: In addition to relieving pain and reducing inflamma-
tion, aspirin also lowers fever by acting on the part of thebrain that regulates temperature. The brain then signalsthe blood vessels to widen, which allows heat to leavethe body more quickly.
Recommended dosage
Adults
TO RELIEVE PAIN OR REDUCE FEVER. one to two
tablets every three to four hours, up to six times per day.
TO REDUCE THE RISK OF STROKE. one tablet four
times a day or two tablets twice a day.
--------------------
Text: reduce the pain of a herpes outbreak and shorten the
period of viral shedding.
Amantadine (Symmetrel), oseltamivir (Tamiflu),
rimantidine (Flumadine), and zanamivir (Relenza) areuseful in treatment of influenza virus. Amantadine,
rimantadine, and oseltamivir may be administeredthroughout the flu season as preventatives for patientswho cannot take influenza virus vaccine.
Cidofovir (Vistide), foscarnet (Foscavir), and ganci-
--------------------
Text: medicine. Anyone who must be exposed to ex

In [17]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [19]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [20]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=pinecone_vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [22]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\rites\anaconda3\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Acne is a skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  Mouth cancer, also known as oral cancer, refers to any type of cancer that affects the mouth, tongue, lips, or throat. The most common types of mouth cancer are squamous cell carcinomas, which account for about 90% of all oral cancers. Other types of mouth cancer include melanoma, adenocarcinoma, and lymphoma. Mouth cancer can be caused by a variety of factors, including tobacco use, heavy alcohol consumption, and exposure to the human papillomavirus (HPV). Symptoms of mouth cancer may include a sore or lesion in the mouth that does not heal, bleeding, pain, and difficulty swallowing. If you suspect you or someone you know may have mouth cancer, it is important to see a doctor or dentist as soon as possible for proper diagnosis and treatment.
